In [1]:
import pandas as pd
from pySheets import DictArray
from IPython.display import HTML

In [6]:
hubStat = pd.read_excel('hubstats.xls')
hs = DictArray(hubStat)[0]

In [2]:
#hs.keys()

In [10]:
users, online, date, time, page = hs.values()
HTML(f"<h3>{page} has {users} users ({online} online) as of {date} {time}</h3>")

In [2]:
df = pd.read_excel('spacestmp.xls')
da = DictArray(df)

In [3]:
FBdf = pd.read_excel('facebook.xls')
FBdf
da.extend(DictArray(FBdf))

In [4]:
kinds = set(df['kind']).union(set(FBdf['kind']))

In [5]:
def sorter(row):
    return row['members'] + row['followers'] + row['posts']

def FBsorter(row):
    return row['members']

tables={}

for kind in kinds:
    if kind=='facebook':
        sortFunc = FBsorter
        followersAndPosts = ''
    else:
        sortFunc = sorter
        followersAndPosts="""        
        <td>followers</td>
        <td>posts</td>
"""
        
    lines=[]
    lines.append('<table>')
    lines.append(f"""
        <tr><td><b>{kind}</b> spaces</td></tr>
        <tr>
        <td>name</td>
        <td>members</td>
        {followersAndPosts}
        </tr>""")

    rows = [row for row in da if row['kind']==kind]
    rows = sorted(rows, key=sortFunc, reverse=True)
    for row in rows:
        href=row['href']
        lines.append(f"<td><a href={href}>{row['name']}</href></td>")
        lines.append(f"<td>{row['members']}</td>")
        if kind != 'facebook':
            lines.append(f"<td>{row['followers']}</td>")
            lines.append(f"<td>{row['posts']}</td>")
        lines.append('</tr>')

    lines.append('</table>')
    tables[kind]='\n'.join(lines)


HTML(f"""
<table>
  <tr> 
      <td style="vertical-align:top">
      {tables['official']}
      </td>
      <td style="vertical-align:top">
      {tables['chapter']}
      </td>
  </tr>
  <tr>
      <td style="vertical-align:top">
      {tables['project']}
      </td>
      <td style="vertical-align:top">
      {tables['volunteer']}
      </td>
      <td style="vertical-align:top">
      {tables['facebook']}
      </td>
  </tr>

  
</table>
""")

In [6]:
videosDF = pd.read_excel('videos.xls')

In [7]:
def vsorter(row):
    return row['views']


#massage the views
videosDA = DictArray(videosDF)
for i,row in enumerate(videosDA):
    views=row['views']
    if type(views) == type('string'):
        views = views.replace(' views','')
        views = views.replace('K','000').replace('.','')
        views = int(views)
    videosDA[i]['views'] = views
    
videosDA = sorted(videosDA, key=vsorter, reverse=True)

In [8]:
# make table
def TableRow(entries):  #entries is a list of strings
    lines=[]
    lines.append('  <tr>')
    for entry in entries:
        lines.append(f"    <td>{entry}</td>".replace('nan',''))
    lines.append('  </tr>')
    return '\n'.join(lines)  #a block of HTML

def Table(TableRows):  #a list of blocks of HTML
    lines=['<table>']
    for TableRow in TableRows:
        lines.append(TableRow)
    lines.append('</table>')
    return '\n'.join(lines) #a full table of HTML

TableRows=[]
TableRows.append(TableRow('Page Age Subscribers Views'.split()))
for row in videosDA:
    page = f"<a href={row['href']}>{row['title']}</a>"
    age = row['age']
    subscribers=row['subscribers']
    views = row['views']
    TableRows.append(TableRow([page, age, subscribers, views]))
    
HTML('<h4>YouTube Video Channel</h4>' + Table(TableRows))

Page,Age,Subscribers,Views
e-NABLE,"Joined Jan 6, 2014",1180.0,169990
e-NABLING Families • 3D Printed Prosthetic Hands,5 years ago,,93000
Hands For Haiti,4 years ago,,75000
Alex's Elbow Driven e-NABLE Arm!,5 years ago,,71000
Look How Far We've Come - Liam's Hands,4 years ago,,51000
e-NABLE Talon Hand 2.0 - Tutorial Part 2,5 years ago,,44000
e-NABLE - Swimmin With The Beast!,5 years ago,,34000
Thank You Liam,5 years ago,,33000
Enabling The Future,4 years ago,,30000
Taking Recipient Photos For e-NABLE,4 years ago,,26000


In [25]:
def Tableize(listOfLists):
    """a list of lists
    each row in the outer list is a row in the table
    rather  like pd.DataFrame(ll)
    """
    return Table([TableRow(row) for row in listOfLists])

In [10]:
ll = [['head','head','head'],
     [1,2,3],
     [4,5,6]]